# Compare simulations with different EBL

In [ ]:
import numpy as np
from pathlib import Path

import matplotlib.pyplot as plt

from pop_plot import col_size, sig_legend, history, plot_historical, redshift, eiso

import setup
import init as init

from utilities import MyLabel, single_legend

plt.style.use('seaborn-talk') # Make the labels readable
#plt.style.use('seaborn-poster') # Make the labels readable - bug with normal x marker !!!

## Define data and tags 

In [ ]:
folders = [ 
    "../../../output/pop_vis24_fullmoonveto-100iter-noacc" ,
    "../../../output/pop_vis24_fullmoonveto-100iter-noacc-franceschini",
    "../../../output/pop_vis24_fullmoonveto-100iter-noacc-finke",
    "../../../output/pop_vis24_fullmoonveto-100iter-noacc-builtin"
]
taglist = ["Dominguez", "Franceschini", "Finke", "Gilmore"]

## Read data - checks

In [ ]:
poplist = []
for tag, f in zip(taglist, folders):
    print("\n",30*"=",tag,30*"=","\n")
    file = init.create_csv(file=None, datafile=Path(f,"data.txt"),debug=False)
    pop = init.get_data(file, debug=False)
    poplist.append(pop)
    
    # Sanity checks
    # (grb, gn0, gs0, gn, gs, gb) = pop   
    # init.sanity_check(file, grb, gn0, gs0, gn, gs, gb, debug=True, old=False)    
    
print("Done !")

### Loop over data and plots


In [ ]:
import pandas as pd
import matplotlib.cm as cm
dump = True
det_level = 90 # INFER IT
nbin = 20

print("Detected 5 sigma 90%CL :")
for pop, tag,  in zip(poplist, taglist):
    (grb, gn0, gs0, gn, gs, gb) = pop
    fullpop = pd.concat([gn0,gs0,gb],axis=0)
    print("   {:15s} = {:3d}".format(tag,len(fullpop[fullpop.d5s>=det_level])))
    if dump:
        file = open(tag+".txt","w+")
        data = fullpop[fullpop.d5s>=det_level].z.values
        print(data)
        file.write(str(data))
        file.close()

In [ ]:
file = open("Dominguez.txt", "r")
content = file.read()
z = np.fromstring(content[1:-1], dtype=np.float, sep=' ')
plt.hist(z)

In [ ]:
x = gb[gb.d5s>=det_level].z.values
x

In [ ]:
# Dump redshifts to text file
pop = fullpop[0]
print(" EBL :",taglist[0]," n (>5s 90%CL) = ",len(pop))
file = open
for g in fullpop[fullpop.d5s>=det_level]

### Redshift

In [ ]:
zmax= 5

ipop=0
first = True
bins = 10
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(20,6),sharey=True)

for ax1, pop, tag,  in zip(ax, poplist, taglist):
    (grb, gn0, gs0, gn, gs, gb) = pop
    fullpop = pd.concat([gn0,gs0,gb],axis=0)
 
    mask = (fullpop.d5s>det_level) & (fullpop.z<=zmax)
    plot = ax1.hist(fullpop[mask].z,bins=bins,alpha=0.7,color="tab:orange",
                edgecolor="grey",lw=2,
            label=MyLabel(fullpop[mask].z,label=tag))
    bins = plot[1]
    ax1.legend()
    ax1.set_xlabel("Redshift")
    ax1.grid(ls=":",alpha=0.5)
    if first:
        first = False
        plot1 = plot
        ax1.set_ylabel("Counts")

    else:
        n, bins, bars = plot1
        ax1.bar(bins[:-1]+0.5*(bins[1:]-bins[:-1]),
                n,
                width=(bins[1:]-bins[:-1]),
                facecolor="none",edgecolor="grey",lw=2,alpha=1)
    ipop+=1

plt.tight_layout()

### Eiso

In [ ]:
Eisomin = 5e50
Eisomax = 5e55

ipop=0
first = True
bins = 10
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(20,6),sharey=True)

for ax1, pop, tag,  in zip(ax, poplist, taglist):
    
    (grb, gn0, gs0, gn, gs, gb) = pop
    fullpop = pd.concat([gn0,gs0,gb],axis=0)
 
    mask = (fullpop.d5s>det_level)# & (fullpop.Eiso>=Eisomin) & (fullpop.Eiso<=Eisomax)
    plot = ax1.hist(np.log10(fullpop[mask].Eiso),bins=bins,
                    alpha=0.7,color="tab:orange",
                    edgecolor="grey",lw=2,
                    label=MyLabel(np.log10(fullpop[mask].Eiso),label=tag))
    bins = plot[1]

    ax1.legend()
    ax1.set_xlabel("$log_{10} \ E_{iso}$")
    ax1.grid(ls=":",alpha=0.5)
    if first:
        first = False
        plot1 = plot
        ax1.set_ylabel("Counts")

    else:
        n, bins, bars = plot1
        print(n)
        ax1.bar(bins[:-1]+0.5*(bins[1:]-bins[:-1]),
                n,
                width=(bins[1:]-bins[:-1]),
                facecolor="none",alpha=1,edgecolor="grey",lw=2,)
    ipop+=1

plt.tight_layout()